In [12]:
%pip install llama-index

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: C:\Users\arnav\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [13]:
%pip install llama-index-llms-openai

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: C:\Users\arnav\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [25]:
import os

os.environ["OPENAI_API_KEY"] = "sk-"

import nest_asyncio
nest_asyncio.apply()

In [15]:
#copied imports from llama-index example for subquestion
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.query_engine import SubQuestionQueryEngine
from llama_index.core.callbacks import CallbackManager, LlamaDebugHandler
from llama_index.core import Settings

#copied imports from llama-index example for multistep
from llama_index.llms.openai import OpenAI
from IPython.display import Markdown, display

In [16]:
# LLM (gpt-3.5)
gpt35 = OpenAI(temperature=0, model="gpt-3.5-turbo")

# LLM (gpt-4)
gpt4 = OpenAI(temperature=0, model="gpt-4")

In [21]:
# import os
from llama_index.core import SimpleDirectoryReader
import urllib.request

os.makedirs('data/paul_graham/', exist_ok=True)

# Download the file
url = 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/paul_graham/paul_graham_essay.txt'
file_path = 'data/paul_graham/paul_graham_essay.txt'
urllib.request.urlretrieve(url, file_path)

# load documents
pg_essay = SimpleDirectoryReader("./data/paul_graham/").load_data()


In [22]:
# Using the LlamaDebugHandler to print the trace of the sub questions
# captured by the SUB_QUESTION callback event type
llama_debug = LlamaDebugHandler(print_trace_on_end=True)
callback_manager = CallbackManager([llama_debug])

Settings.callback_manager = callback_manager

In [23]:
index = VectorStoreIndex.from_documents(documents)

**********
Trace: index_construction
    |_CBEventType.EMBEDDING ->  0.481921 seconds
**********


In [26]:
# build index and query engine
vector_query_engine = VectorStoreIndex.from_documents(
    pg_essay,
    use_async=True,
).as_query_engine()

**********
Trace: index_construction
    |_CBEventType.EMBEDDING ->  1.739059 seconds
**********


In [28]:
# setup base query engine as tool
query_engine_tools = [
    QueryEngineTool(
        query_engine=vector_query_engine,
        metadata=ToolMetadata(
            name="pg_essay",
            description="Paul Graham essay on What I Worked On",
        ),
    ),
]

query_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=query_engine_tools,
    use_async=True,
)

In [71]:
llama_debug.flush_event_logs() #flush for easier retrieval of subquestions

response = query_engine.query(
    "Summarize the essay, and discuss the authors life before and after the essay was written."
) #todo: figure out how to stop it from printing responses

Generated 3 sub questions.
[pg_essay] Q: Summarize the essay 'What I Worked On' by Paul Graham
[pg_essay] Q: Discuss Paul Graham's life before writing the essay 'What I Worked On'
[pg_essay] Q: Discuss Paul Graham's life after writing the essay 'What I Worked On'
[pg_essay] A: Paul Graham continued to write essays through 2020, but he also began considering other projects to work on. He reflected on how he had chosen his past projects and wrote an essay exploring this topic in detail. Graham found the answer to be lengthy and complex, leading him to believe that others might find it interesting and encouraging, especially those with similarly intricate life experiences. This exploration of his decision-making process marked a shift in his focus from solely writing essays to contemplating new endeavors beyond essay writing.
[pg_essay] A: The essay "What I Worked On" by Paul Graham discusses his journey of discovering the importance of writing essays online and how it influenced his work

In [79]:
from llama_index.core.callbacks import CBEventType

event_pairs = llama_debug.get_event_pairs(CBEventType.SUB_QUESTION)

sub_questions = []
for pair in event_pairs:
    sub_q_object = str(pair[0].payload["sub_question"])
    start_index = sub_q_object.find("sub_question=")
    end_index = sub_q_object.find(", tool_name=")
    if start_index == -1 or end_index == -1:
        # smth wrong
        continue

    new_q = sub_q_object[start_index+14:end_index-1]
    sub_questions.append(new_q)

# probably a better way to do this than to substring

    

sub_question="Summarize the essay 'What I Worked On' by Paul Graham
sub_question="Discuss Paul Graham's life before writing the essay 'What I Worked On'
sub_question="Discuss Paul Graham's life after writing the essay 'What I Worked On'


In [81]:
from llama_index.core.indices.query.query_transform.base import (
    StepDecomposeQueryTransform,
)

# gpt-4
step_decompose_transform = StepDecomposeQueryTransform(llm=gpt4, verbose=True)

# gpt-3
step_decompose_transform_gpt3 = StepDecomposeQueryTransform(llm=gpt35, verbose=True)

index_summary = "Used to answer questions about the author"

In [84]:
from llama_index.core.query_engine import MultiStepQueryEngine

mqe = VectorStoreIndex.from_documents(documents).as_query_engine(llm=gpt4)

multi_query_engine = MultiStepQueryEngine(
    query_engine=mqe,
    query_transform=step_decompose_transform,
    index_summary=index_summary,
)

for sub_q in sub_questions:
    print("query: ", sub_q)
    response = multi_query_engine.query(sub_q)
    # print(response)

**********
Trace: index_construction
    |_CBEventType.EMBEDDING ->  0.319597 seconds
**********
query:  Summarize the essay 'What I Worked On' by Paul Graham
> Current query: Summarize the essay 'What I Worked On' by Paul Graham
> New query: Who is the author of the essay 'What I Worked On'?
> Current query: Summarize the essay 'What I Worked On' by Paul Graham
> New query: What are the main points of the essay 'What I Worked On' by Paul Graham?
> Current query: Summarize the essay 'What I Worked On' by Paul Graham
> New query: What are the key experiences and realizations Paul Graham shares in his essay 'What I Worked On'?
**********
Trace: query
    |_CBEventType.QUERY ->  24.958391 seconds
      |_CBEventType.TEMPLATING ->  0.0 seconds
      |_CBEventType.LLM ->  0.824248 seconds
      |_CBEventType.QUERY ->  1.12308 seconds
        |_CBEventType.RETRIEVE ->  0.120999 seconds
          |_CBEventType.EMBEDDING ->  0.105321 seconds
        |_CBEventType.SYNTHESIZE ->  0.999274 second